# NB 2

Given that the linear probability models gave rather weak results, I want to slowly build up to running an instrumental variable (2SLS) regression by first creating some control variables and assessing the assumptions of an IV/2SLS Model.

In [1]:
# Set Up
import pandas as pd
import numpy as np

# These lines make warnings look nicer
import warnings
warnings.simplefilter('ignore', FutureWarning)

# Regression
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from IPython.core.display import HTML
from statsmodels.sandbox.regression.gmm import IV2SLS

# Graphing
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (10,10)
import seaborn as sns

# Specialty Imports

For the IV regression I'm going to be using statsmodels implementation of IV/2-Stage Least Squares. Documentation can be found here https://www.statsmodels.org/stable/generated/statsmodels.sandbox.regression.gmm.IV2SLS.html

In [2]:
sample = pd.read_csv('sample.csv')

# Part II: Using instrumental variables

Using the story from the Notion page, we established that a couple's desire to have both a boy and a girl is a strong incentive for them to have more kids. In order for us to use "samesex" as an instrument, we will want to truly consider the claim that gender assignment is completely random because only then will we satisfy the exogeneity assumptions of instrumental varaibles.

Read more here: https://en.wikipedia.org/wiki/Instrumental_variables_estimation


Let's first begin by controlling for the father's influence on having more children by looking at the father's race and education:

$$ y_i = \beta_0 + \beta_1<ageDummies> + \beta_2<educDummies> + \beta_3<agefstmDummies> + \beta_4 aged +\beta_5 educd + \beta_6 blackm + \beta_7 hispm + \beta_8 othracem + \epsilon_i$$





In [3]:
educDummies = pd.get_dummies(sample['educm'], prefix = "ed")
ageDummies = pd.get_dummies(sample['agem'], prefix = "age")
age_fstmDummies = pd.get_dummies(sample['agefstm'], prefix = "age_ftsm")


m3_features = pd.concat([ageDummies, educDummies, age_fstmDummies], axis = 1)

Now we can test the importance of the father's race and their education confounding. Notice that our excluded group here are white, non-hispanic mothers.

In [4]:
new_features = sample[['aged', 'educd', 'blackm', 'hispm', 'othracem']]
noDad = sample[['blackm', 'hispm', 'othracem']]



In [5]:
m4_features = pd.concat([m3_features, new_features], axis = 1)
m4_features_noDad = pd.concat([m3_features, noDad], axis = 1)
            

m4_outcome = sample[['morekids']]
np.mean(m4_outcome)

morekids    0.373101
dtype: float64

Let's conduct a joint significance test for Model 4a which includes the fatherly race variables.


$$H_0: \beta_{blackm}= 0 , \beta_{hispm} = 0, \beta_{educd} = 0$$

$$H_1 =   \beta_{blackm} \neq 0, \beta_{hispm} \neq 0, beta_{educd} \neq 0$$

In [8]:
m4_a = sm.OLS(m4_outcome ,sm.add_constant(m4_features))
m4_a = m4_a.fit()
m4_a.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               morekids   R-squared:                       0.078
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     350.6
Date:                Sun, 10 May 2020   Prob (F-statistic):               0.00
Time:                        14:05:18   Log-Likelihood:            -1.5415e+05
No. Observations:              236459   AIC:                         3.084e+05
Df Residuals:                  236401   BIC:                         3.090e+05
Df Model:                          57                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.2315      0.024      9.695      0.000       0.185       0.278
age_21         -0.2321      0.013    -18.467      0.000      -0.257      -0.207
age_22         -0.1983      0.009    -22.365      0.000      -0.216      -0.181
age_23         -0.1655      0.007    -23.846      0.000      -0.179      -0.152
age_24         -0.1104      0.006    -19.169      0.000      -0.122      -0.099
age_25         -0.0622      0.005    -12.364      0.000      -0.072      -0.052
age_26         -0.0382      0.005     -8.460      0.000      -0.047      -0.029
age_27          0.0043      0.004      1.041      0.298      -0.004       0.013
age_28          0.0257      0.004      6.441      0.000       0.018       0.033
age_29          0.0648      0.004     16.706      0.000       0.057       0.072
age_30          0.0858      0.004     22.446      0.000       0.078       0.093
age_31          0.1153      0.004     29.986      0.000       0.108       0.123
age_32          0.1453      0.004     37.155      0.000       0.138       0.153
age_33          0.1680      0.004     41.270      0.000       0.160       0.176
age_34          0.2003      0.005     44.486      0.000       0.191       0.209
age_35          0.2286      0.005     48.470      0.000       0.219       0.238
ed_0            0.1300      0.019      6.880      0.000       0.093       0.167
ed_1            0.0876      0.044      2.009      0.045       0.002       0.173
ed_2            0.2290      0.029      7.906      0.000       0.172       0.286
ed_3            0.1900      0.022      8.681      0.000       0.147       0.233
ed_4            0.1068      0.018      6.025      0.000       0.072       0.142
ed_5            0.1276      0.016      8.141      0.000       0.097       0.158
ed_6            0.1041      0.010      9.972      0.000       0.084       0.125
ed_7            0.0653      0.011      5.942      0.000       0.044       0.087
ed_8            0.0357      0.007      4.897      0.000       0.021       0.050
ed_9            0.0045      0.006      0.726      0.468      -0.008       0.017
ed_10          -0.0186      0.006     -3.260      0.001      -0.030      -0.007
ed_11          -0.0307      0.006     -5.236      0.000      -0.042      -0.019
ed_12          -0.0904      0.004    -22.953      0.000      -0.098      -0.083
ed_13          -0.0834      0.005    -17.111      0.000      -0.093      -0.074
ed_14          -0.0739      0.005    -14.509      0.000      -0.084      -0.064
ed_15          -0.0519      0.006     -8.000      0.000      -0.065      -0.039
ed_16          -0.0570      0.005    -10.952      0.000      -0.067      -0.047
ed_17          -0.0906      0.007    -12.142      0.000      -0.105      -0.076
ed_18          -0.1276      0.009    -14.025      0.000      -0.145      -0.110
ed_19          -0.0942      0.018     -5.18

The p-value for the $aged$ and $educd$ are $p = 0.164, 0.661$ respectively which indicate the fatherly variables are not significantin determining the probability of returning. However we observe that the race variables have p-values of 0 which indicate significant determinants.

Let's run another regression taking those out.


In [9]:
m4_b = sm.OLS(m4_outcome ,sm.add_constant(m4_features_noDad))
m4_b = m4_b.fit()
m4_b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               morekids   R-squared:                       0.078
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     363.3
Date:                Sun, 10 May 2020   Prob (F-statistic):               0.00
Time:                        14:05:19   Log-Likelihood:            -1.5415e+05
No. Observations:              236459   AIC:                         3.084e+05
Df Residuals:                  236403   BIC:                         3.090e+05
Df Model:                          55                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -7.022e+10   1.78e+11     -0.394      0.694    -4.2e+11    2.79e+11
age_21       1.067e+08   1.89e+10      0.006      0.995   -3.69e+10    3.72e+10
age_22       1.067e+08   1.89e+10      0.006      0.995   -3.69e+10    3.72e+10
age_23       1.067e+08   1.89e+10      0.006      0.995   -3.69e+10    3.72e+10
age_24       1.067e+08   1.89e+10      0.006      0.995   -3.69e+10    3.72e+10
age_25       1.067e+08   1.89e+10      0.006      0.995   -3.69e+10    3.72e+10
age_26       1.067e+08   1.89e+10      0.006      0.995   -3.69e+10    3.72e+10
age_27       1.067e+08   1.89e+10      0.006      0.995   -3.69e+10    3.72e+10
age_28       1.067e+08   1.89e+10      0.006      0.995   -3.69e+10    3.72e+10
age_29       1.067e+08   1.89e+10      0.006      0.995   -3.69e+10    3.72e+10
age_30       1.067e+08   1.89e+10      0.006      0.995   -3.69e+10    3.72e+10
age_31       1.067e+08   1.89e+10      0.006      0.995   -3.69e+10    3.72e+10
age_32       1.067e+08   1.89e+10      0.006      0.995   -3.69e+10    3.72e+10
age_33       1.067e+08   1.89e+10      0.006      0.995   -3.69e+10    3.72e+10
age_34       1.067e+08   1.89e+10      0.006      0.995   -3.69e+10    3.72e+10
age_35       1.067e+08   1.89e+10      0.006      0.995   -3.69e+10    3.72e+10
ed_0          4.81e+10   1.15e+11      0.417      0.677   -1.78e+11    2.74e+11
ed_1          4.81e+10   1.15e+11      0.417      0.677   -1.78e+11    2.74e+11
ed_2          4.81e+10   1.15e+11      0.417      0.677   -1.78e+11    2.74e+11
ed_3          4.81e+10   1.15e+11      0.417      0.677   -1.78e+11    2.74e+11
ed_4          4.81e+10   1.15e+11      0.417      0.677   -1.78e+11    2.74e+11
ed_5          4.81e+10   1.15e+11      0.417      0.677   -1.78e+11    2.74e+11
ed_6          4.81e+10   1.15e+11      0.417      0.677   -1.78e+11    2.74e+11
ed_7          4.81e+10   1.15e+11      0.417      0.677   -1.78e+11    2.74e+11
ed_8          4.81e+10   1.15e+11      0.417      0.677   -1.78e+11    2.74e+11
ed_9          4.81e+10   1.15e+11      0.417      0.677   -1.78e+11    2.74e+11
ed_10         4.81e+10   1.15e+11      0.417      0.677   -1.78e+11    2.74e+11
ed_11         4.81e+10   1.15e+11      0.417      0.677   -1.78e+11    2.74e+11
ed_12         4.81e+10   1.15e+11      0.417      0.677   -1.78e+11    2.74e+11
ed_13         4.81e+10   1.15e+11      0.417      0.677   -1.78e+11    2.74e+11
ed_14         4.81e+10   1.15e+11      0.417      0.677   -1.78e+11    2.74e+11
ed_15         4.81e+10   1.15e+11      0.417      0.677   -1.78e+11    2.74e+11
ed_16         4.81e+10   1.15e+11      0.417      0.677   -1.78e+11    2.74e+11
ed_17         4.81e+10   1.15e+11      0.417      0.677   -1.78e+11    2.74e+11
ed_18         4.81e+10   1.15e+11      0.417      0.677   -1.78e+11    2.74e+11
ed_19         4.81e+10   1.15e+11      0.41

Notice that the Adj. R-squared values for both regressions are 0.078 which indicate that omitting the dad variables are insignficant when determining the likelihood of having more kids. Let's continue testing our larger hypothesis that same sex children is a decent _exogenous_ determinant of the family size.

## Adding samesex and applying that to Model 4 which has the dummies, aged, educd, and race

In [10]:
m4c_features = pd.concat([m4_features, sample['samesex']], axis = 1)

In [11]:
m4_c = sm.OLS(m4_outcome ,sm.add_constant(m4c_features))
m4_c = m4_c.fit()
m4_c.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               morekids   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     369.0
Date:                Mon, 25 May 2020   Prob (F-statistic):               0.00
Time:                        17:33:56   Log-Likelihood:            -1.5350e+05
No. Observations:              236459   AIC:                         3.071e+05
Df Residuals:                  236400   BIC:                         3.077e+05
Df Model:                          58                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.1994      0.024      8.371      0.000       0.153       0.246
age_21         -0.2358      0.013    -18.814      0.000      -0.260      -0.211
age_22         -0.2011      0.009    -22.752      0.000      -0.218      -0.184
age_23         -0.1678      0.007    -24.249      0.000      -0.181      -0.154
age_24         -0.1127      0.006    -19.614      0.000      -0.124      -0.101
age_25         -0.0642      0.005    -12.790      0.000      -0.074      -0.054
age_26         -0.0406      0.005     -9.019      0.000      -0.049      -0.032
age_27          0.0028      0.004      0.671      0.502      -0.005       0.011
age_28          0.0236      0.004      5.926      0.000       0.016       0.031
age_29          0.0628      0.004     16.243      0.000       0.055       0.070
age_30          0.0842      0.004     22.093      0.000       0.077       0.092
age_31          0.1137      0.004     29.649      0.000       0.106       0.121
age_32          0.1434      0.004     36.765      0.000       0.136       0.151
age_33          0.1664      0.004     40.978      0.000       0.158       0.174
age_34          0.1985      0.004     44.207      0.000       0.190       0.207
age_35          0.2264      0.005     48.114      0.000       0.217       0.236
ed_0            0.1266      0.019      6.720      0.000       0.090       0.164
ed_1            0.0869      0.043      2.001      0.045       0.002       0.172
ed_2            0.2272      0.029      7.868      0.000       0.171       0.284
ed_3            0.1883      0.022      8.624      0.000       0.145       0.231
ed_4            0.1082      0.018      6.122      0.000       0.074       0.143
ed_5            0.1227      0.016      7.847      0.000       0.092       0.153
ed_6            0.1025      0.010      9.839      0.000       0.082       0.123
ed_7            0.0643      0.011      5.875      0.000       0.043       0.086
ed_8            0.0341      0.007      4.686      0.000       0.020       0.048
ed_9            0.0034      0.006      0.539      0.590      -0.009       0.016
ed_10          -0.0195      0.006     -3.413      0.001      -0.031      -0.008
ed_11          -0.0318      0.006     -5.436      0.000      -0.043      -0.020
ed_12          -0.0921      0.004    -23.428      0.000      -0.100      -0.084
ed_13          -0.0849      0.005    -17.469      0.000      -0.094      -0.075
ed_14          -0.0754      0.005    -14.859      0.000      -0.085      -0.065
ed_15          -0.0530      0.006     -8.191      0.000      -0.066      -0.040
ed_16          -0.0588      0.005    -11.328      0.000      -0.069      -0.049
ed_17          -0.0914      0.007    -12.286      0.000      -0.106      -0.077
ed_18          -0.1291      0.009    -14.230      0.000      -0.147      -0.111
ed_19          -0.0954      0.018     -5.26

### The average effect of having the first two children be the same sex is the beta coeffecient 0.0688, that is to say that if a couple has two kids of the same sex, on average they are 6% more likely to have a 3rd child.


Let's evaluate the claim that families only care about have 1 son. This would mean that if we included the variables $boys_2$ in our model then  $\beta_{boys2}$ would be statistically insignificant because the couple already has a son. 

$$ H_0: \beta_{boys2} = 0$$
$$ H_1: \beta_{boys2} \neq 0$$

Then we will test if having 2 girls increases the probability of having more kids. I expect the $\beta_{girls2}$ to not only be significant but having a weight greater than 6% because of the (outdated) cultural norms of _"needing to have a male heir"_.

$$ H_0: \beta_{boys2} = 0.06$$
$$ H_1: \beta_{boys2} > 0.06$$

In [12]:
m4c_2boys_features = pd.concat([m4c_features, sample['boys2']], axis = 1)

m4c_2boys = sm.OLS(m4_outcome ,sm.add_constant(m4c_2boys_features))
m4c_2boys = m4c_2boys.fit()
m4c_2boys.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               morekids   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     363.9
Date:                Mon, 25 May 2020   Prob (F-statistic):               0.00
Time:                        17:33:57   Log-Likelihood:            -1.5347e+05
No. Observations:              236459   AIC:                         3.071e+05
Df Residuals:                  236399   BIC:                         3.077e+05
Df Model:                          59                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.1987      0.024      8.341      0.000       0.152       0.245
age_21         -0.2359      0.013    -18.820      0.000      -0.260      -0.211
age_22         -0.2012      0.009    -22.756      0.000      -0.218      -0.184
age_23         -0.1679      0.007    -24.264      0.000      -0.181      -0.154
age_24         -0.1127      0.006    -19.629      0.000      -0.124      -0.101
age_25         -0.0643      0.005    -12.810      0.000      -0.074      -0.054
age_26         -0.0407      0.005     -9.034      0.000      -0.050      -0.032
age_27          0.0028      0.004      0.669      0.504      -0.005       0.011
age_28          0.0235      0.004      5.916      0.000       0.016       0.031
age_29          0.0628      0.004     16.250      0.000       0.055       0.070
age_30          0.0841      0.004     22.074      0.000       0.077       0.092
age_31          0.1136      0.004     29.624      0.000       0.106       0.121
age_32          0.1434      0.004     36.764      0.000       0.136       0.151
age_33          0.1664      0.004     40.985      0.000       0.158       0.174
age_34          0.1984      0.004     44.192      0.000       0.190       0.207
age_35          0.2263      0.005     48.106      0.000       0.217       0.236
ed_0            0.1268      0.019      6.729      0.000       0.090       0.164
ed_1            0.0857      0.043      1.973      0.048       0.001       0.171
ed_2            0.2270      0.029      7.860      0.000       0.170       0.284
ed_3            0.1884      0.022      8.629      0.000       0.146       0.231
ed_4            0.1084      0.018      6.131      0.000       0.074       0.143
ed_5            0.1226      0.016      7.844      0.000       0.092       0.153
ed_6            0.1025      0.010      9.839      0.000       0.082       0.123
ed_7            0.0645      0.011      5.887      0.000       0.043       0.086
ed_8            0.0341      0.007      4.688      0.000       0.020       0.048
ed_9            0.0034      0.006      0.552      0.581      -0.009       0.016
ed_10          -0.0195      0.006     -3.415      0.001      -0.031      -0.008
ed_11          -0.0319      0.006     -5.451      0.000      -0.043      -0.020
ed_12          -0.0920      0.004    -23.423      0.000      -0.100      -0.084
ed_13          -0.0849      0.005    -17.472      0.000      -0.094      -0.075
ed_14          -0.0754      0.005    -14.848      0.000      -0.085      -0.065
ed_15          -0.0531      0.006     -8.210      0.000      -0.066      -0.040
ed_16          -0.0587      0.005    -11.311      0.000      -0.069      -0.049
ed_17          -0.0913      0.007    -12.276      0.000      -0.106      -0.077
ed_18          -0.1291      0.009    -14.234      0.000      -0.147      -0.111
ed_19          -0.0951      0.018     -5.24

$p_{boys2}$ is 0 and $\beta_{boys2} = -0.0213$ which means that having two boys decreases the probability of having more children by about 2% which is in the direction of our alternative hypothesis!

In [14]:
m4c_2girls_features = pd.concat([m4c_features, sample['girls2']], axis = 1)

m4c_2girls = sm.OLS(m4_outcome ,sm.add_constant(m4c_2girls_features))
m4c_2girls = m4c_2girls.fit()
m4c_2girls.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               morekids   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     363.9
Date:                Mon, 25 May 2020   Prob (F-statistic):               0.00
Time:                        17:36:13   Log-Likelihood:            -1.5347e+05
No. Observations:              236459   AIC:                         3.071e+05
Df Residuals:                  236399   BIC:                         3.077e+05
Df Model:                          59                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.1987      0.024      8.341      0.000       0.152       0.245
age_21         -0.2359      0.013    -18.820      0.000      -0.260      -0.211
age_22         -0.2012      0.009    -22.756      0.000      -0.218      -0.184
age_23         -0.1679      0.007    -24.264      0.000      -0.181      -0.154
age_24         -0.1127      0.006    -19.629      0.000      -0.124      -0.101
age_25         -0.0643      0.005    -12.810      0.000      -0.074      -0.054
age_26         -0.0407      0.005     -9.034      0.000      -0.050      -0.032
age_27          0.0028      0.004      0.669      0.504      -0.005       0.011
age_28          0.0235      0.004      5.916      0.000       0.016       0.031
age_29          0.0628      0.004     16.250      0.000       0.055       0.070
age_30          0.0841      0.004     22.074      0.000       0.077       0.092
age_31          0.1136      0.004     29.624      0.000       0.106       0.121
age_32          0.1434      0.004     36.764      0.000       0.136       0.151
age_33          0.1664      0.004     40.985      0.000       0.158       0.174
age_34          0.1984      0.004     44.192      0.000       0.190       0.207
age_35          0.2263      0.005     48.106      0.000       0.217       0.236
ed_0            0.1268      0.019      6.729      0.000       0.090       0.164
ed_1            0.0857      0.043      1.973      0.048       0.001       0.171
ed_2            0.2270      0.029      7.860      0.000       0.170       0.284
ed_3            0.1884      0.022      8.629      0.000       0.146       0.231
ed_4            0.1084      0.018      6.131      0.000       0.074       0.143
ed_5            0.1226      0.016      7.844      0.000       0.092       0.153
ed_6            0.1025      0.010      9.839      0.000       0.082       0.123
ed_7            0.0645      0.011      5.887      0.000       0.043       0.086
ed_8            0.0341      0.007      4.688      0.000       0.020       0.048
ed_9            0.0034      0.006      0.552      0.581      -0.009       0.016
ed_10          -0.0195      0.006     -3.415      0.001      -0.031      -0.008
ed_11          -0.0319      0.006     -5.451      0.000      -0.043      -0.020
ed_12          -0.0920      0.004    -23.423      0.000      -0.100      -0.084
ed_13          -0.0849      0.005    -17.472      0.000      -0.094      -0.075
ed_14          -0.0754      0.005    -14.848      0.000      -0.085      -0.065
ed_15          -0.0531      0.006     -8.210      0.000      -0.066      -0.040
ed_16          -0.0587      0.005    -11.311      0.000      -0.069      -0.049
ed_17          -0.0913      0.007    -12.276      0.000      -0.106      -0.077
ed_18          -0.1291      0.009    -14.234      0.000      -0.147      -0.111
ed_19          -0.0951      0.018     -5.24

$p_{girls2}$ is 0 and $\beta_{girls2} = 0.0213$ which means that having two girls increases the likelihood of having more children which can confirm our hypothesis based on cultural norms. Adjusted R-squared fell by about .002 compared to the regression with 2 boys, so roughly speaking they have about the same predictive power.